In [3]:
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid", {
 'axes.spines.bottom': True,
 'axes.spines.left': True,
 'axes.spines.right': True,
 'axes.spines.top': True
})
sns.set(font_scale=1)
#sns.set_style("darkgrid")
#sns.set_context("poster")
import sys
import os
from scipy.optimize import newton, minimize, fsolve
from scipy.interpolate import InterpolatedUnivariateSpline, krogh_interpolate, PchipInterpolator, interp1d
import numpy as np
import copy
import re
from pathlib import Path

import neutcurve
from neutcurve.colorschemes import CBMARKERS, CBPALETTE

import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import matplotlib.colors as mcolors
palette = list(mcolors.TABLEAU_COLORS.keys())
palette.extend(['salmon', 'teal','yellowgreen'])
%matplotlib inline

import random
from pyswarm import pso
import isocor

In [4]:
element_natural_prob = {
    'C': 0.0107,
    'H': 0.000115,
    'N': 0.00364
}

In [5]:
### Adjust for specific label:
def obj_iso(v1, element_natural_prob, element_list, print_sim=False):
    # Adjust:
    element_prob = [element_natural_prob[element_list[0]], v1]

    N_trials = 10000000
    ra = np.random.binomial(1, element_prob, (N_trials, len(element_prob)))
    sim_data = np.bincount(ra.sum(1)) / len(ra) * 100
    # Adjust:
    loss = abs(obs_data[0] - sim_data[0]) + abs(obs_data[1] - sim_data[1]) + abs(obs_data[2] - sim_data[2])
    print(v1, loss)
    if print_sim:
        return(sim_data)
    else:
        return(loss)

In [9]:
# Gln 15N1. m+0,1,2 abundance data:
metabolite = 'Gln'
ion_formula = 'C5H11N2O3'
tracer_element = '15N'
charge = 1
obs_data = {
    0: 0.2383330,
    1: 99.4840964,
    2: 0.2775706
}
element_list = ['N']*2

In [10]:
# Correct the effect of natural abundance of other elements than carbon.
# No natural abundance correction for the tracer and assume 100% tracer purity
# to avoid multiplication with the purity matrix.
corrector_HR = isocor.mscorrectors.MetaboliteCorrectorFactory(formula=ion_formula, label=metabolite, tracer=tracer_element, tracer_purity=[0, 1], resolution=240000, mz_of_resolution=200, charge=charge, correct_NA_tracer=False, resolution_formula_code='orbitrap')

In [11]:
# Calculate the corrected isotope fraction:
corrected_area, iso_fraction, res, m_enr = corrector_HR.correct([obs_data[i] for i in range(len(obs_data))])

In [15]:
# Update the data:
for i in range(len(obs_data)):
    obs_data[i] = iso_fraction[i] * 100

In [16]:
# Use particle swarm to find best probability to fit the data:
def iso_fun(v1): return(obj_iso(v1, element_natural_prob, element_list))
xopt, fopt = pso(iso_fun, (0.95,), (0.9999999,), swarmsize=20, maxiter=20, minstep=1e-7, minfunc=1e-7)

[0.95871071] 7.8841128
[0.97120877] 5.404852800000008
[0.95999894] 7.652812799999997
[0.99321314] 1.033992799999997
[0.95929484] 7.770152800000003
[0.97774699] 4.100772800000008
[0.98653423] 2.3674127999999914
[0.98412888] 2.8542528000000065
[0.98764943] 2.1585728000000013
[0.99828443] 0.17167879999999675
[0.97642781] 4.353972800000005
[0.9913469] 1.4333927999999896
[0.96256956] 7.122012799999998
[0.9704022] 5.5783727999999915
[0.9509103] 9.419892799999994
[0.95896026] 7.858312799999997
[0.99054367] 1.5609927999999904
[0.98855957] 1.970212800000007
[0.96551119] 6.542732799999992
[0.99346044] 0.9964128000000012
[0.9558917] 8.439252799999995
[0.95667352] 8.296012800000002
[0.9853465] 2.605952800000006
[0.97548295] 4.5545727999999945
[0.97977246] 3.7073128000000057
[0.96890257] 5.873672800000004
[0.98858077] 1.947412799999998
[0.98448996] 2.7754528000000054
[0.97473288] 4.715892800000003
[0.98830001] 2.0121727999999868
[0.98680279] 2.322192799999991
[0.9999999] 0.47664599999999624
[0.95] 

[0.99731491] 0.2308728000000076
[0.99814855] 0.1775787999999887
[0.997745] 0.16749879999998984
[0.99760159] 0.1688527999999985
[0.99781983] 0.17745880000000092
[0.99790249] 0.17689879999999783
[0.99757151] 0.17657280000000153
[0.99754875] 0.1876127999999916
[0.99795243] 0.17207879999999767
[0.99761449] 0.16961879999998994
[0.99819532] 0.17243879999998818
[0.99778593] 0.17501879999999126
[0.99762703] 0.1700187999999974
[0.99776798] 0.16541880000000456
[0.99741516] 0.2118127999999953
[0.99737182] 0.21197280000000657
[0.99772906] 0.1696788000000058
[0.99761379] 0.17633879999999794
[0.99774923] 0.1662588000000011
[0.99778389] 0.17409879999998995
[0.99756343] 0.18713280000000068
[0.99802363] 0.1694787999999969
[0.99763295] 0.17169879999999707
[0.99760829] 0.16587879999999308
[0.99783109] 0.17371880000000017
[0.99795707] 0.16407880000001004
[0.99760592] 0.17759280000000716
[0.99767619] 0.1702788000000068
[0.99804832] 0.17025879999999016
[0.99773568] 0.17393879999999798
[0.99781801] 0.1654187

In [17]:
# Print probability:
xopt[0], fopt

(0.9977744756308705, 0.1580387999999986)

In [18]:
# Get predictions:
obj_iso(xopt[0], element_natural_prob, element_list, print_sim=True)

0.9977744756308705 0.1711187999999918


array([ 0.22536, 99.41151,  0.36313])